<a href="https://colab.research.google.com/github/pragmalingu/experiments/blob/master/00_Data/LISACorpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup an Elasticsearch Instance in Google Colab

Everthing to connect to Elasticsearch, for detailed explaination see [this Notebook.](https://)
Download:

In [ ]:
import os
from subprocess import Popen, PIPE, STDOUT
# download elasticsearch
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.1-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.9.1-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.9.1

Start a local server:

In [ ]:
# start server
es_server = Popen(['elasticsearch-7.9.1/bin/elasticsearch'], 
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )
# client-side
!pip install elasticsearch -q
from elasticsearch import Elasticsearch
from datetime import datetime
es = Elasticsearch()
es.ping()  # got True

# Parsing LISA Corpus

You can get the corpus from [this link](http://ir.dcs.gla.ac.uk/resources/test_collections/lisa/).  <br>
For detailed information about the format of the files, see the PragmaLingu [Data Sets](https://pragmalingu.de/docs/guides/data-comparison). 
You can learn about parsing in general by reading our [parsing guide](https://pragmalingu.de/docs/guides/how-to-parse).

### Dowlnoad Data

Download and unzip data.

In [ ]:
!wget http://ir.dcs.gla.ac.uk/resources/test_collections/lisa/lisa.tar.gz
!tar -xf lisa.tar.gz

Set paths to the dowloaded data as variables:

In [ ]:
PATH_TO_LISA_TXT = '/content/'
PATH_TO_LISA_QRY = '/content/LISA.QUE'
PATH_TO_LISA_REL = '/content/LISARJ.NUM'

### Imports

Make all the imports:

In [ ]:
from collections import defaultdict
import re
import json
from io import StringIO
import numpy as np
import os

### Process Data

Since the documents of the LISA corpus are splitted in several files, we have to get all the files first:

In [ ]:
file_regex = re.compile('LISA[0-5]')
lisa_files = [i for i in os.listdir(PATH_TO_LISA_TXT) if os.path.isfile(os.path.join(PATH_TO_LISA_TXT,i)) and \
         re.match(file_regex,i)]

Get the text entries from the text and query file preprocessed as a list. Query and text files have diffrent entry markers. Since there are more than one text file, we need to iterate over them while reading them:

In [ ]:
txt_entry_marker = re.compile('\*{44}',re.MULTILINE)
qry_entry_marker = re.compile('#')

def get_data(PATH_TO_FILES, marker):
  """
  Reads multiple files and spilts text into entries at the entry marker.
  The 'marker' contains the regex at which we want to split
  Pops last element since it's empty.
  """
  with open (PATH_TO_FILES,'r') as f:
    text = f.read().replace('     ','')
    lines = re.split(marker,text)
    lines.pop()
  return lines

lisa_txt_list = []
for name in lisa_files: 
  lisa_txt_list.extend(get_data(PATH_TO_LISA_TXT+name, txt_entry_marker))

lisa_qry_list = get_data(PATH_TO_LISA_QRY, qry_entry_marker)

The entries in the text files contain only information on the ID, the title and the text. To obtain the relevant information we have to strip the entries from 'Documents' to get the ID as the first element. 

Afterwards we can split the entries using the newlines. The first splitted element is the ID, the title can be stored in 1 or more entries until an empty element follows. Everything after that is the text.

In [ ]:
doc_strip = re.compile('\n?Document {1,2}')

lisa_txt_list_stripped = []
lisa_txt_data = defaultdict(dict)

for el in lisa_txt_list:
  lisa_txt_list_stripped.append(re.sub(doc_strip,'', el))

for entry in lisa_txt_list_stripped:
  parts = entry.split('\n')
  empty_index = parts.index('')
  ID = parts[0].strip(' ')
  title = parts[1:empty_index]
  text = parts[empty_index+1:]
  lisa_txt_data[ID]['title'] = ''.join(title)
  lisa_txt_data[ID]['text'] = ''.join(text)

The query file is easily parsed by splitting the elements at the newline marker and saving the second splitted element as ID while the following elements are the question:

In [ ]:
# process the query data
lisa_qry_data = defaultdict(dict)

# first line is a special case because it doesn't start with a newline
first_line = lisa_qry_list[0]
first_question = first_line[1:]
lisa_qry_data[int(first_line[0])]['question'] = first_question

# after that every line can be handle the same way
for n in range(0,len(lisa_qry_list)-1):
  line = re.split('\n',lisa_qry_list[n+1])
  question = ' '.join(line[2:])
  lisa_qry_data[int(line[1])]['question'] = question

Relevance assesments are saved as numpy and parsed to dictionary:

In [ ]:
# process relevance assesments without rating
lisa_rel = defaultdict(list)

with open (PATH_TO_LISA_REL,'r') as f:
  file = f.read().strip('       ').replace('\n','')
  lines = re.split(' ',file)
  lines = list(filter(None, lines))
  n = 0
  while n < len(lines):
    ID = int(lines[n])
    num_rel = int(lines[n+1])
    rels = lines[(n+2):(n+num_rel+2)]
    lisa_rel[ID].extend(rels)
    n = n+1+num_rel+1

### Create index for LISA corpus

Create an index for the LISA corpus. This is only possible if it isn't created yet.

(For more information see the [Elasticsearch documentation](https://elasticsearch-py.readthedocs.io/en/master/api.html#elasticsearch.client.IndicesClient.create))

In [ ]:
lisa_index = "lisa-corpus"

es.indices.create(lisa_index)

create_response = es.cat.indices()
print(create_response)

Index all the documents that are processed to the created index in elasticsearch:

(For more information see the [Elasticsearch documentation](https://elasticsearch-py.readthedocs.io/en/master/#example-usage))

In [ ]:
for ID, doc_data in lisa_txt_data.items():
  es.index(index=lisa_index, id=ID, body=doc_data)

Verify if everthing went right by printing it:

In [ ]:
create_response = es.cat.indices()
print(create_response)

green  open porter-medline-corpus        pCZngrxjSS2FwFgxbUqNnA 1 0  1032   0     1mb     1mb
green  open snowball-cranfield-corpus    W6kIploWQJGNMm6xwGtJbg 1 0  1400   0   1.5mb   1.5mb
yellow open security-auditlog-2020.07.26 3KQWvUwlRHOYj3AeOYcfkg 1 1    31   0  75.6kb  75.6kb
yellow open security-auditlog-2020.07.27 C3A4S_lhSC-y5NEvhhhAAg 1 1    20   0  36.9kb  36.9kb
yellow open security-auditlog-2020.07.28 sLtcImDYTmafKHDoxhmg8g 1 1    22   0  72.3kb  72.3kb
yellow open security-auditlog-2020.07.29 s_wFvB3PTeKWfVSZFdlgOw 1 1     9   0 123.3kb 123.3kb
green  open snowball-medline-corpus      OpgXBcIrRnma8hWRj3YeSg 1 0  1032   0     1mb     1mb
green  open porter-index                 en25T-o8Tduy5B0rCQnBrg 1 0     1   0   4.5kb   4.5kb
yellow open pragmalingu-cisi-corpus      CH3y6fjrR-6dlstL1FwRAQ 1 1     0   0    208b    208b
green  open stemmer-cacm-corpus          xDyo3niKQ2GPnqOwqeB1YQ 1 0  3204   0     4mb     4mb
green  open stemming-time-corpus         RntweNy1Rl2zhkwcpEL

### Use Corpus in Ranking API

Use the ranking evaluation API from elasticsearch to evaluate the corpus:

(For more information see the [python documentation](https://elasticsearch-py.readthedocs.io/en/master/api.html?highlight=_rank_eval#elasticsearch.Elasticsearch.rank_eval) and the [Elasticsearch documentation](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-rank-eval.html#search-rank-eval))

In [ ]:
lisa_index = "lisa-corpus"

def create_query_body(query_dict, rel_dict, index_name):
  """
  The function creates a request for every query in query_dict and rates the relevant documents with rel_dict to 1.
  The index name has to be the same as from the documents your looking at.
  An evaluation body for the elasticsearch ranking API is returned.
  """
  eval_body = {
      "requests":'',
      "metric": {
          "recall": {
              "relevant_rating_threshold": 1,
              "k": 20
              }
      }
  }
  requests = [] 
  current_request = defaultdict(lambda: defaultdict())
  current_rel = {"_index": index_name, "_id": '', "rating": int}
  for query_ID, query_txt in query_dict.items():
    current_query = {"query": { "multi_match": { "query": '' , "fields" : ["title","text"]}}}
    current_query["query"]["multi_match"]["query"] = query_txt['question']
    current_request["id"] = 'Query_'+str(query_ID)
    current_request["request"] = current_query.copy()
    current_request["ratings"] = [{"_index": index_name, "_id": str(el), "rating": 1} for el in rel_dict[query_ID]]
    requests.append(current_request.copy())
  eval_body["requests"] = requests
  return eval_body

lisa_create = create_query_body(lisa_qry_data, lisa_rel, lisa_index)

lisa_eval_body = json.dumps(lisa_create)

Print results of Ranking API:

In [ ]:
lisa_res = es.rank_eval(lisa_eval_body,lisa_index)

print(json.dumps(lisa_res, indent=4, sort_keys=True))

{
    "details": {
        "Query_1": {
            "hits": [
                {
                    "hit": {
                        "_id": "3350",
                        "_index": "lisa-corpus",
                        "_score": 522.3177,
                        "_type": "_doc"
                    },
                    "rating": null
                },
                {
                    "hit": {
                        "_id": "5767",
                        "_index": "lisa-corpus",
                        "_score": 515.72644,
                        "_type": "_doc"
                    },
                    "rating": null
                },
                {
                    "hit": {
                        "_id": "3604",
                        "_index": "lisa-corpus",
                        "_score": 505.27722,
                        "_type": "_doc"
                    },
                    "rating": null
                },
                {
                    "hit": {
 